**MOVIE VIEWER BEHAVIOR ANALYSIS**

In [2]:
!pip install catboost

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

In [4]:
watch_logs = pd.read_csv("watch_logs.csv")
users = pd.read_csv("users.csv")
movies = pd.read_csv("movies.csv")
devices = pd.read_csv("devices.csv")

In [5]:
watch_logs

,log_id,user_id,movie_id,device_id,watch_date,watch_duration,watch_time_slot,liked,rating_given,completed
0,1,6563,1464,2984,2024-11-28,57,Afternoon,No,1,False
1,2,19479,1074,315,2024-09-23,18,Afternoon,No,5,False
2,3,5274,1792,2340,2025-04-03,175,Evening,Yes,3,True
3,4,7934,882,2097,2024-08-21,146,Night,Yes,1,True
4,5,10395,868,2541,2024-04-03,168,Night,Yes,3,True
...,...,...,...,...,...,...,...,...,...,...
19995,19996,29268,1798,1057,2023-12-20,175,Night,No,2,True
19996,19997,11757,1123,2710,2024-07-26,32,Morning,No,2,False
19997,19998,24432,26,2758,2024-09-28,42,Night,No,1,False
19998,19999,29825,1978,1656,2025-02-19,98,Evening,Yes,1,False


In [6]:
users

,user_id,name,age,gender,country,signup_date,membership_type,preferred_genre
0,1,Kenneth Quinn,62,Female,Iran,2022-08-06,Premium,Thriller
1,2,Pamela Grant,12,Female,Saint Pierre and Miquelon,2025-03-25,Free,Sci-Fi
2,3,Danny Weaver,29,Other,United States of America,2023-03-29,Premium,Sci-Fi
3,4,Anita Davis,65,Female,Fiji,2022-11-25,VIP,Horror
4,5,Todd Higgins,12,Female,Kenya,2020-10-10,Free,Action
...,...,...,...,...,...,...,...,...
39995,39996,Michael Mason,54,Other,Antarctica (the territory South of 60 deg S),2024-11-22,Premium,Romance
39996,39997,David Sosa,61,Other,Dominica,2023-07-09,Free,Drama
39997,39998,Taylor Williams,63,Female,Kazakhstan,2024-01-12,Free,Thriller
39998,39999,Anna Vasquez,33,Other,Slovakia (Slovak Republic),2021-05-24,Premium,Horror


In [7]:
#Create marged Dataframe
df = watch_logs.merge(users, on="user_id", how="left") \
               .merge(movies, on="movie_id", how="left") \
               .merge(devices, on="device_id", how="left")

In [8]:
df

,log_id,user_id,movie_id,device_id,watch_date,watch_duration,watch_time_slot,liked,rating_given,completed,...,release_year,duration_mins,language,age_rating,avg_rating,device_type,os,brand,screen_size_inch,supports_hd
0,1,6563,1464,2984,2024-11-28,57,Afternoon,No,1,False,...,2000,135,Japanese,R,4.03,Tablet,macOS,Apple,56.1,True
1,2,19479,1074,315,2024-09-23,18,Afternoon,No,5,False,...,2008,96,Japanese,PG,1.47,Laptop,Android,Apple,7.2,False
2,3,5274,1792,2340,2025-04-03,175,Evening,Yes,3,True,...,1981,135,German,NC-17,2.92,Mobile,Linux,Dell,29.2,False
3,4,7934,882,2097,2024-08-21,146,Night,Yes,1,True,...,2005,154,Hindi,G,3.99,Smart TV,macOS,HP,12.6,False
4,5,10395,868,2541,2024-04-03,168,Night,Yes,3,True,...,1989,137,English,PG,2.90,Smart TV,macOS,HP,47.3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,29268,1798,1057,2023-12-20,175,Night,No,2,True,...,1980,131,French,PG-13,1.49,Laptop,iOS,HP,50.9,True
19996,19997,11757,1123,2710,2024-07-26,32,Morning,No,2,False,...,2018,174,German,R,3.47,Mobile,iOS,Sony,5.6,True
19997,19998,24432,26,2758,2024-09-28,42,Night,No,1,False,...,2012,157,Hindi,NC-17,4.65,Tablet,macOS,LG,29.0,True
19998,19999,29825,1978,1656,2025-02-19,98,Evening,Yes,1,False,...,2008,171,French,NC-17,1.86,Mobile,Android,LG,30.4,False


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   log_id            20000 non-null  int64  
 1   user_id           20000 non-null  int64  
 2   movie_id          20000 non-null  int64  
 3   device_id         20000 non-null  int64  
 4   watch_date        20000 non-null  object 
 5   watch_duration    20000 non-null  int64  
 6   watch_time_slot   20000 non-null  object 
 7   liked             20000 non-null  object 
 8   rating_given      20000 non-null  int64  
 9   completed         20000 non-null  bool   
 10  name              20000 non-null  object 
 11  age               20000 non-null  int64  
 12  gender            20000 non-null  object 
 13  country           20000 non-null  object 
 14  signup_date       20000 non-null  object 
 15  membership_type   20000 non-null  object 
 16  preferred_genre   20000 non-null  object

In [10]:
df.columns

Index(['log_id', 'user_id', 'movie_id', 'device_id', 'watch_date',
       'watch_duration', 'watch_time_slot', 'liked', 'rating_given',
       'completed', 'name', 'age', 'gender', 'country', 'signup_date',
       'membership_type', 'preferred_genre', 'title', 'genre', 'release_year',
       'duration_mins', 'language', 'age_rating', 'avg_rating', 'device_type',
       'os', 'brand', 'screen_size_inch', 'supports_hd'],
      dtype='object')

In [11]:
# Drop unnecessary columns
df.drop(columns=["log_id", "user_id", "movie_id", "device_id", "name", "title", "country", "signup_date", "brand"], inplace=True)


In [12]:
df

,watch_date,watch_duration,watch_time_slot,liked,rating_given,completed,age,gender,membership_type,preferred_genre,genre,release_year,duration_mins,language,age_rating,avg_rating,device_type,os,screen_size_inch,supports_hd
0,2024-11-28,57,Afternoon,No,1,False,38,Female,Premium,Documentary,Sci-Fi,2000,135,Japanese,R,4.03,Tablet,macOS,56.1,True
1,2024-09-23,18,Afternoon,No,5,False,58,Female,VIP,Action,Sci-Fi,2008,96,Japanese,PG,1.47,Laptop,Android,7.2,False
2,2025-04-03,175,Evening,Yes,3,True,54,Other,Free,Documentary,Horror,1981,135,German,NC-17,2.92,Mobile,Linux,29.2,False
3,2024-08-21,146,Night,Yes,1,True,13,Other,Premium,Documentary,Romance,2005,154,Hindi,G,3.99,Smart TV,macOS,12.6,False
4,2024-04-03,168,Night,Yes,3,True,33,Female,VIP,Horror,Thriller,1989,137,English,PG,2.90,Smart TV,macOS,47.3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2023-12-20,175,Night,No,2,True,34,Male,Premium,Thriller,Comedy,1980,131,French,PG-13,1.49,Laptop,iOS,50.9,True
19996,2024-07-26,32,Morning,No,2,False,22,Other,Free,Horror,Comedy,2018,174,German,R,3.47,Mobile,iOS,5.6,True
19997,2024-09-28,42,Night,No,1,False,15,Other,VIP,Drama,Sci-Fi,2012,157,Hindi,NC-17,4.65,Tablet,macOS,29.0,True
19998,2025-02-19,98,Evening,Yes,1,False,61,Male,Free,Comedy,Sci-Fi,2008,171,French,NC-17,1.86,Mobile,Android,30.4,False


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   watch_date        20000 non-null  object 
 1   watch_duration    20000 non-null  int64  
 2   watch_time_slot   20000 non-null  object 
 3   liked             20000 non-null  object 
 4   rating_given      20000 non-null  int64  
 5   completed         20000 non-null  bool   
 6   age               20000 non-null  int64  
 7   gender            20000 non-null  object 
 8   membership_type   20000 non-null  object 
 9   preferred_genre   20000 non-null  object 
 10  genre             20000 non-null  object 
 11  release_year      20000 non-null  int64  
 12  duration_mins     20000 non-null  int64  
 13  language          20000 non-null  object 
 14  age_rating        20000 non-null  object 
 15  avg_rating        20000 non-null  float64
 16  device_type       20000 non-null  object

In [14]:
df.isna().sum()

,0
watch_date,0
watch_duration,0
watch_time_slot,0
liked,0
rating_given,0
completed,0
age,0
gender,0
membership_type,0
preferred_genre,0


In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
#Label Encoding
categorical_cols = df.select_dtypes(include="object").columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [17]:
df

,watch_date,watch_duration,watch_time_slot,liked,rating_given,completed,age,gender,membership_type,preferred_genre,genre,release_year,duration_mins,language,age_rating,avg_rating,device_type,os,screen_size_inch,supports_hd
0,526,57,0,0,1,False,38,0,1,2,6,2000,135,4,4,4.03,3,4,56.1,True
1,460,18,0,0,5,False,58,0,2,0,6,2008,96,4,2,1.47,0,0,7.2,False
2,652,175,1,1,3,True,54,2,0,2,4,1981,135,2,1,2.92,1,1,29.2,False
3,427,146,3,1,1,True,13,2,1,2,5,2005,154,3,0,3.99,2,4,12.6,False
4,287,168,3,1,3,True,33,0,2,4,7,1989,137,0,2,2.90,2,4,47.3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,182,175,3,0,2,True,34,1,1,7,1,1980,131,1,3,1.49,0,3,50.9,True
19996,401,32,2,0,2,False,22,2,0,4,1,2018,174,2,4,3.47,1,3,5.6,True
19997,465,42,3,0,1,False,15,2,2,3,6,2012,157,3,1,4.65,3,4,29.0,True
19998,609,98,1,1,1,False,61,1,0,1,6,2008,171,1,1,1.86,1,0,30.4,False


In [18]:
df['age'].fillna(df['age'].mean(), inplace=True)

/tmp/ipython-input-1503503937.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(df['age'].mean(), inplace=True)


In [19]:
df.columns

Index(['watch_date', 'watch_duration', 'watch_time_slot', 'liked',
       'rating_given', 'completed', 'age', 'gender', 'membership_type',
       'preferred_genre', 'genre', 'release_year', 'duration_mins', 'language',
       'age_rating', 'avg_rating', 'device_type', 'os', 'screen_size_inch',
       'supports_hd'],
      dtype='object')

In [20]:
# Features (X) - select multiple columns
x = df[['watch_date', 'watch_duration', 'watch_time_slot',
        'rating_given', 'completed', 'age', 'gender', 'membership_type',
        'preferred_genre', 'genre', 'release_year', 'duration_mins', 'language',
        'age_rating', 'avg_rating', 'device_type', 'os', 'screen_size_inch',
        'supports_hd']]

# Target (y) - select a single column
y = df['liked']


In [21]:
x

,watch_date,watch_duration,watch_time_slot,rating_given,completed,age,gender,membership_type,preferred_genre,genre,release_year,duration_mins,language,age_rating,avg_rating,device_type,os,screen_size_inch,supports_hd
0,526,57,0,1,False,38,0,1,2,6,2000,135,4,4,4.03,3,4,56.1,True
1,460,18,0,5,False,58,0,2,0,6,2008,96,4,2,1.47,0,0,7.2,False
2,652,175,1,3,True,54,2,0,2,4,1981,135,2,1,2.92,1,1,29.2,False
3,427,146,3,1,True,13,2,1,2,5,2005,154,3,0,3.99,2,4,12.6,False
4,287,168,3,3,True,33,0,2,4,7,1989,137,0,2,2.90,2,4,47.3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,182,175,3,2,True,34,1,1,7,1,1980,131,1,3,1.49,0,3,50.9,True
19996,401,32,2,2,False,22,2,0,4,1,2018,174,2,4,3.47,1,3,5.6,True
19997,465,42,3,1,False,15,2,2,3,6,2012,157,3,1,4.65,3,4,29.0,True
19998,609,98,1,1,False,61,1,0,1,6,2008,171,1,1,1.86,1,0,30.4,False


In [22]:
y

,liked
0,0
1,0
2,1
3,1
4,1
...,...
19995,0
19996,0
19997,0
19998,1


In [23]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [24]:
#Model Creation
knn_model=KNeighborsClassifier()
nb_model=GaussianNB()
lr_model=LogisticRegression()
rf_model=RandomForestClassifier()
svm_model=SVC()
cat_model = CatBoostClassifier()

In [25]:
knn_model.fit(x_train,y_train)
nb_model.fit(x_train,y_train)
lr_model.fit(x_train,y_train)
rf_model.fit(x_train,y_train)
svm_model.fit(x_train,y_train)
cat_model.fit(x_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Learning rate set to 0.033658
0:	learn: 0.6929724	total: 71.1ms	remaining: 1m 11s
1:	learn: 0.6927623	total: 93ms	remaining: 46.4s
2:	learn: 0.6924355	total: 107ms	remaining: 35.5s
3:	learn: 0.6921534	total: 125ms	remaining: 31.1s
4:	learn: 0.6919736	total: 144ms	remaining: 28.7s
5:	learn: 0.6917810	total: 164ms	remaining: 27.2s
6:	learn: 0.6915849	total: 185ms	remaining: 26.2s
7:	learn: 0.6913665	total: 206ms	remaining: 25.6s
8:	learn: 0.6911745	total: 226ms	remaining: 24.8s
9:	learn: 0.6910054	total: 245ms	remaining: 24.2s
10:	learn: 0.6908242	total: 265ms	remaining: 23.8s
11:	learn: 0.6906288	total: 286ms	remaining: 23.5s
12:	learn: 0.6904546	total: 304ms	remaining: 23.1s
13:	learn: 0.6902988	total: 324ms	remaining: 22.8s
14:	learn: 0.6901535	total: 343ms	remaining: 22.5s
15:	learn: 0.6899670	total: 361ms	remaining: 22.2s
16:	learn: 0.6898306	total: 381ms	remaining: 22s
17:	learn: 0.6895935	total: 402ms	remaining: 22s
18:	learn: 0.6894243	total: 424ms	remaining: 21.9s
19:	learn: 0.6

In [26]:
y_pred_knn=knn_model.predict(x_test)
y_pred_nb=nb_model.predict(x_test)
y_pred_lr=lr_model.predict(x_test)
y_pred_rf=rf_model.predict(x_test)
y_pred_svm=svm_model.predict(x_test)
y_pred_cat=cat_model.predict(x_test)

In [27]:
accuracy_knn=accuracy_score(y_test,y_pred_knn)
accuracy_knn

0.502

In [28]:
accuracy_nb=accuracy_score(y_test,y_pred_nb)
accuracy_nb

0.5065

In [29]:
accuracy_lr=accuracy_score(y_test,y_pred_lr)
accuracy_lr

0.49925

In [30]:
accuracy_rf=accuracy_score(y_test,y_pred_rf)
accuracy_rf

0.49475

In [31]:
accuracy_svm=accuracy_score(y_test,y_pred_svm)
accuracy_svm

0.50525

In [32]:
accuracy_cat=accuracy_score(y_test,y_pred_cat)
accuracy_cat

0.50275

In [33]:
#Best Result from Catboost

In [34]:
#Creating pkl file

import pickle

with open('catboost_model.pkl', 'wb') as file:
    pickle.dump(cat_model, file)
